In [1]:
#import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
tot_df = []
for j,i in enumerate(np.arange(2016, 2021, 1)):
for j,i in enumerate(np.arange(2016, 2016, 1)):\n",
"    tot_df.append(pd.read_csv('../Data/Raw/CDC/CDCSTDcountysexrace'+ \n",
"                              str(i) +'.csv', sep = ',', skiprows = 4))\n",
"# df = pd.concat(tot_df, axis = 0)\n",
"df = pd.read_csv('../Data/Raw/CDC/CDCSTDcountysexrace2016.csv', skiprows = 4)"